In [1]:
!conda list langchain

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(182): Could not remove or rename /langchain_env/conda-meta/packaging-24.0-pyhd8ed1ab_0.json.  Please remove this file manually (you may need to reboot to free file handles)
# packages in environment at /langchain_env:
#
# Name                    Version                   Build  Channel
langchain                 0.1.13                   pypi_0    pypi
langchain-community       0.0.29                   pypi_0    pypi
langchain-core            0.1.36                   pypi_0    pypi
langchain-experimental    0.0.55                   pypi_0    pypi
langchain-openai          0.1.1                    pypi_0    pypi
langchain-text-splitters  0.0.1                    pypi_0    pypi


# 1.OpenAI Function Calling

In [2]:
import openai
from openai import OpenAI
import os
openai.__version__

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [3]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [4]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [5]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [6]:
import os

chat_completion = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
    functions=functions,
    function_call = "none"
)

In [7]:
chat_completion

ChatCompletion(id='chatcmpl-98OGGnWJrb081Ypdu6tKDPpRolIot', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I can help you with that. Let me check the current weather in Boston for you.', role='assistant', function_call=None, tool_calls=None))], created=1711786912, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b28b39ffa8', usage=CompletionUsage(completion_tokens=18, prompt_tokens=83, total_tokens=101))

In [8]:
message = chat_completion.choices[0].message

In [9]:
message.function_call

# 2. LangChain Expression Language (LCEL)

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

#### Simple chain

In [11]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [12]:
chain = prompt | model | output_parser

In [13]:
chain.invoke({"topic": "chicken"})

'Why did the chicken join a band? Because it had the drumsticks!'

#### Link a more complex chain language expression

In [14]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [15]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

/langchain_env/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [16]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [17]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [18]:
from langchain.schema.runnable import RunnableMap

In [19]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [20]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [21]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [22]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

#### Bind

In [28]:
functions = [
    {
        "name": "pet_care",
        "description": "Get information about caring for a pet",
        "parameters": {
            "type": "object",
            "properties": {
                "pet_care_type": {
                    "type": "string",
                    "description": "The type care you want to know about, e.g. feeding, grooming",
                },
            },
        },
        "required": ["pet_care_type"],
    }
]

In [29]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

model = ChatOpenAI(temperature=0).bind(functions=functions)

In [30]:
runnable = prompt | model

In [31]:
runnable.invoke({"input": "I want to know how to feed my pet"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"pet_care_type":"feeding"}', 'name': 'pet_care'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 77, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'function_call', 'logprobs': None})

#### Fallbacks

In [47]:
from langchain_openai import ChatOpenAI
import json

In [83]:
simple_model = ChatOpenAI(
    temperature=0,
    max_tokens=1000,
    model="gpt-3.5-turbo"
)

simple_chain = simple_model | StrOutputParser() | json.loads
simple_chain_error = simple_model | json.loads

In [84]:
challenge = "write three authors who wrote about the american revolution in a json blob"

In [86]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content=challenge),
]

simple_chain_with_fallback = simple_chain_error.with_fallbacks([simple_chain])
authors = simple_chain_with_fallback.invoke(messages)
print(authors)

{'authors': ['David McCullough', 'Joseph J. Ellis', 'Gordon S. Wood']}


#### OpenAI Function Calling

Some function calling, definition

In [2]:
from typing import List, Dict
from pydantic import BaseModel, Field

class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [5]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

wather_search_openai_function = convert_pydantic_to_openai_function(WeatherSearch)
wather_search_openai_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

Model building

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI().bind(functions=[wather_search_openai_function])

Input prompt

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", "You're a helpful assistant"),
    ("user", "{input}")]
)

Create our chain based on LCEL

In [12]:
chain = prompt | model 

Chain execution

In [14]:
chain.invoke({"input", "What's the weather like at J.F. Kenedy?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 86, 'total_tokens': 103}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'function_call', 'logprobs': None})

#### Tagging and extraction example definition

In [16]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function
from pydantic import BaseModel, Field
from typing import List, Dict


class Person(BaseModel):
    """A function that allows to obtain the person's name, age, and hobbies."""
    name: str = Field(description="The person's name")
    age: int = Field(description="The person's age")
    hobbies: List[str] = Field(description="The person's hobbies")

class People(BaseModel):
    """A function that allows to obtain a list of people's names, ages, and hobbies."""
    people: List[Person] = Field(description="List of people's names, ages, and hobbies")



people_tagging_function = convert_pydantic_to_openai_function(People)
people_tagging_function

{'name': 'People',
 'description': "A function that allows to obtain a list of people's names, ages, and hobbies.",
 'parameters': {'$defs': {'Person': {'description': "A function that allows to obtain the person's name, age, and hobbies.",
    'properties': {'name': {'description': "The person's name",
      'type': 'string'},
     'age': {'description': "The person's age", 'type': 'integer'},
     'hobbies': {'description': "The person's hobbies",
      'items': {'type': 'string'},
      'type': 'array'}},
    'required': ['name', 'age', 'hobbies'],
    'type': 'object'}},
  'properties': {'people': {'description': "List of people's names, ages, and hobbies",
    'items': {'description': "A function that allows to obtain the person's name, age, and hobbies.",
     'properties': {'name': {'description': "The person's name",
       'type': 'string'},
      'age': {'description': "The person's age", 'type': 'integer'},
      'hobbies': {'description': "The person's hobbies",
       'ite

Model definition

In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI().bind(
    functions=[people_tagging_function])

Prompt input

In [55]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", "You're a helpful assistant"),
    ("human", "{input}")]
)

Output list parser

In [56]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

output_parser = JsonKeyOutputFunctionsParser(key_name="people")

Chain definition

The **regular_chain** just obtain from one review the data of my friends

In [57]:
regular_chain = prompt | model | output_parser

In [58]:
regular_chain.invoke({"input": "my friend is John, he is 25 years old and he likes to play basketball"})

[{'name': 'John', 'age': 25, 'hobbies': ['play basketball']}]

The **chain**, the final chain, obtain the data of my friends given list of reviews

In [59]:
from langchain.schema.runnable import RunnableLambda

prep = RunnableLambda(
    lambda x: [{"input": review} for review in x]
)

In [60]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [61]:
chain = prep | regular_chain.map() | flatten

In [62]:
friends_reviews = [
    "I met John, who is 25 years old and likes to play soccer.",
    "One of my best friends is Mary, who is 30 years old and enjoys reading books.",
    "The first friend I made in college was Alex, who is 22 years old and loves to play video games."
]

friends = chain.invoke(friends_reviews)
print(friends)

[{'name': 'John', 'age': 25, 'hobbies': ['play soccer']}, {'name': 'Mary', 'age': 30, 'hobbies': ['reading books']}, {'name': 'Alex', 'age': 22, 'hobbies': ['play video games']}]


In [65]:
friends = People(people=[Person(**friend) for friend in friends])
friends

People(people=[Person(name='John', age=25, hobbies=['play soccer']), Person(name='Mary', age=30, hobbies=['reading books']), Person(name='Alex', age=22, hobbies=['play video games'])])